In [1]:
import numpy as np
from xlrd import open_workbook
import xlwt
from itertools import product

In [2]:
table1_len = [4, 4, 5, 4, 2, 3, 3]
N1 = len(table1_len)

table2_len = [4, 10, 3, 3, 4, 4]
N2 = len(table2_len)

precision = 3

In [3]:
def read_stage_correlations(table1_len, file_name):
    """reading correlations"""
    
    N1 = len(table1_len)

    correlations = []
    for i in xrange(N1):
        l1 = []
        for j in xrange(table1_len[i]):
            l2 = []
            for k in xrange(N1):
                l3 = []
                for l in xrange(table1_len[k]):
                    l3.append(0)
                l2.append(l3)
            l1.append(l2)
        correlations.append(l1)

    wb = open_workbook(file_name)
    sheets = wb.sheets()

    for factor_row in xrange(1, N1):
        for alternative_row in xrange(table1_len[factor_row]):
            for factor_column in xrange(factor_row):
                for alternative_column in xrange(table1_len[factor_column]):
                    value = sheets[0].cell_value(2+sum(table1_len[1:factor_row])+alternative_row, 
                                                2+sum(table1_len[:factor_column])+alternative_column)
                    if value != '':
                        correlations[factor_row][alternative_row][factor_column][alternative_column] = \
                            correlations[factor_column][alternative_column][factor_row][alternative_row] = float(value)

    print "Correlations: ", correlations
    return correlations

In [4]:
def read_interstage_correlations(table1_len, table2_len, file_name):
    """reading correlations"""
    
    N1 = len(table1_len)
    N2 = len(table2_len)

    correlations = []
    for i in xrange(N1):
        l1 = []
        for j in xrange(table1_len[i]):
            l2 = []
            for k in xrange(N2):
                l3 = []
                for l in xrange(table2_len[k]):
                    l3.append(0)
                l2.append(l3)
            l1.append(l2)
        correlations.append(l1)

    wb = open_workbook(file_name)
    sheets = wb.sheets()

    for factor_row in xrange(N1):
        for alternative_row in xrange(table1_len[factor_row]):
            for factor_column in xrange(N2):
                for alternative_column in xrange(table2_len[factor_column]):
                    value = sheets[0].cell_value(2+sum(table1_len[:factor_row])+alternative_row, 
                                                2+sum(table2_len[:factor_column])+alternative_column)
                    if value != '':
                        correlations[factor_row][alternative_row][factor_column][alternative_column] = float(value)

    print "Interstage correlations: ", correlations
    return correlations

In [5]:
def read_prior_probs(table1_len, file_name):
    """reading prior probabilities"""
    
    N1 = len(table1_len)
    
    prior_probs = []
    for i in xrange(N1):
        prior_probs.append([])

    wb = open_workbook(file_name)
    sheets = wb.sheets()
    for factor in xrange(N1):
        for alternative in xrange(table1_len[factor]):
            prior_probs[factor].append(float(sheets[0].cell_value(1+alternative, factor)))
        temp = sum(prior_probs[factor])
        for alternative in xrange(table1_len[factor]):
            prior_probs[factor][alternative] /= temp

    print "Prior probabilities: ", prior_probs
    return prior_probs

In [6]:
def calculate_conditional_probabilities(table1_len, correlations, prior_probs, file_name):
    """calculating conditional probabilities"""
    N1 = len(table1_len)

    P_ = np.zeros(shape = table1_len+[N1])
    P = np.zeros(shape = table1_len+[N1])

    lists = [np.arange(table1_len[i]) for i in xrange(N1)] + [np.arange(N1)]

    for item in product(*lists):
        first_product = np.prod([prior_probs[j][item[j]] for j in xrange(N1)])/prior_probs[item[-1]][item[item[-1]]]
        second_product = np.prod([np.prod([1+correlations[i][item[i]][j][item[j]] for j in xrange(i+1, N1)])
                                  for i in xrange(N1-1)])
        P_[item] = first_product*second_product

    for item in product(*lists):
        temp = sum([P_[item1] if (item1[-1] == item[-1] and item1[item[-1]] == item[item[-1]]) else 0
                                for item1 in product(*lists)])
        if temp != 0:
            P[item] = P_[item]/temp
        else:
            P[item] = 0

    book = xlwt.Workbook()
    sh = book.add_sheet(u'Умовні ймовірності')
    
    sh.write(0, 0, u'Конфігурація')
    sh.write(0, N1, u'Імовірність за умови відповідної альтернативи кожного з параметрів')
    for i in xrange(N1):
        sh.write(1, i, 'F'+str(i+1))
        sh.write(1, i+N1, 'F'+str(i+1))
    
    row = 2
    for item in product(*[np.arange(table1_len[i]) for i in xrange(N1)]):
        col = 0
        for i in item:
            sh.write(row, col, i+1)
            col += 1
        for j in xrange(N1):
            sh.write(row, j+N1, round(P[item+(j,)], precision+3))
        row +=1
    book.save(file_name+'.xls')
    #print "Conditional probabilities", P
    return P

In [7]:
def form_P_matrix(table1_len, P, i):
    """form matrix P_i"""
    
    N1 = len(table1_len)
    
    j = i+1 if i<N1-1 else 0
    Pi = np.zeros(shape=(table1_len[i], table1_len[j]))
    
    lists = [np.arange(table1_len[k]) for k in xrange(N1)] + [np.arange(N1)]
    for k in xrange(Pi.shape[0]):
        for l in xrange(Pi.shape[1]):
            Pi[k][l] = np.sum([P[item] if (item[-1] == j and item[i] == k and item[j] == l) else 0
                            for item in product(*lists)])
    return Pi

In [8]:
def solve_p(table1_len, Pi, i):
    "calculate probabilities for i-th factor"
    
    Pi = np.roll(Pi, -i)
    Matr = reduce(lambda x, y: np.dot(x, y), Pi)
    res_eig = np.linalg.eig(Matr)
    ind = min(range(len(res_eig[0])), key=lambda i: abs(res_eig[0][i]-1.0))
    return np.real(res_eig[1][:,ind]/sum(res_eig[1][:,ind]))

In [9]:
def calculate_conditional_efficiency(table1_len, table2_len, correlations):
    """calculating conditional efficiency"""
    N1 = len(table1_len)
    N2 = len(table2_len)

    R_ = np.zeros(shape = table1_len+[N2, max(table2_len)])
    R = np.zeros(shape = table1_len+[N2, max(table2_len)])

    lists = [np.arange(table1_len[i]) for i in xrange(N1)] + [np.arange(N2)]

    for item in product(*lists):
        for i in xrange(table2_len[item[-1]]):
            first_product = 1.0/table2_len[item[-1]]
            second_product = np.prod([1+correlations[j][item[j]][item[-1]][i] for j in xrange(N1)])
            R_[item+(i,)] = first_product*second_product

    for item in product(*lists):
        for i in xrange(table2_len[item[-1]]):
            temp = sum([R_[item+(k,)] for k in xrange(table2_len[item[-1]])])
            if temp != 0:
                R[item+(i,)] = R_[item+(i,)]/temp
            else:
                R[item+(i,)] = 0

    #print "Conditional efficiency", R
    return R

In [10]:
def print_Pi_to_excel(Pi, file_name):
    book = xlwt.Workbook()
    sh = book.add_sheet(u'Матриці P_i')
    
    col = 0
    for i in xrange(len(Pi)):
        sh.write(0, col, u'P_'+str(i+1))
        for j in xrange(Pi[i].shape[0]):
            for k in xrange(Pi[i].shape[1]):
                sh.write(j+1, col+k, round(Pi[i][j][k], precision))
        col += Pi[i].shape[1]+1
    book.save(file_name+'.xls')

In [18]:
def print_new_probs_to_excel(p_new, file_name):
    book = xlwt.Workbook()
    sh = book.add_sheet(u'Нові ймовірності')
    
    for i in xrange(len(p_new)):
        sh.write(0, i, 'F'+str(i+1))
        for j in xrange(len(p_new[i])):
            sh.write(1+j, i, round(p_new[i][j], precision))
            
    book.save(file_name+'.xls')

In [12]:
def calculate_first_stage():
    """recalculate probabilities for the 1st stage"""
    
    correlations = read_stage_correlations(table1_len, 'correlations1.xlsx')
    prior_probs = read_prior_probs(table1_len, 'prior_probs1.xlsx')
    P = calculate_conditional_probabilities(table1_len, correlations, prior_probs, 'conditional_probs1')
    Pi = [form_P_matrix(table1_len, P, i) for i in xrange(N1)]
    print_Pi_to_excel(Pi, 'Pi_matrices1')
    print "P_i matrices: ", Pi
    p_new = [solve_p(table1_len, Pi, i) for i in xrange(N1)]
    print_new_probs_to_excel(p_new, 'new_probs1')
    print "Calculated probabilities: ", p_new
    return p_new

In [13]:
def calculate_second_stage(prior_probs):
    """recalculate probabilities for the 2st stage"""
    
    correlations = read_stage_correlations(table2_len, 'correlations2.xlsx')
    P = calculate_conditional_probabilities(table2_len, correlations, prior_probs, 'conditional_probs2')
    Pi = [form_P_matrix(table2_len, P, i) for i in xrange(N2)]
    print_Pi_to_excel(Pi, 'Pi_matrices2')
    print "P_i matrices: ", Pi
    p_new = [solve_p(table2_len, Pi, i) for i in xrange(N2)]
    print_new_probs_to_excel(p_new, 'new_efficiencies2')
    print "Calculated efficiencies: ", p_new
    return p_new

In [14]:
def calculate_efficiency(post_probs1, file_name):
    """calculate efficiency (prior probabilities) for the 2nd table"""
    
    N1 = len(table1_len)
    N2 = len(table2_len)
    
    correlations = read_interstage_correlations(table1_len, table2_len, 'correlations12.xlsx')
    R_cond = calculate_conditional_efficiency(table1_len, table2_len, correlations)
    
    lists = [np.arange(table1_len[i]) for i in xrange(N1)]
    
    R = []
    for i in xrange(N2):
        R.append([])
        for j in xrange(table2_len[i]):
            R[i].append(sum([R_cond[item + (i, j,)]*np.prod([post_probs1[k][item[k]] for k in xrange(N1)])
                             for item in product(*lists)]))
    
    book = xlwt.Workbook()
    sh = book.add_sheet(u'Очікувані результативності')
    
    for i in xrange(N2):
        sh.write(0, i, 'F'+str(i+1))
        for j in xrange(len(R[i])):
            sh.write(1+j, i, round(R[i][j], precision))
            
    book.save(file_name+'.xls')
    print "Efficiency R: ", R
    return R

In [15]:
post_probs1 = calculate_first_stage()

Correlations:  [[[[0, 0, 0, 0], [1.0, -1.0, -1.0, -1.0], [0, 0, 0, 0, 0], [0, 0, 0, 0], [0.8, 0], [0, 0, 0.5], [0, 0, 0]], [[0, 0, 0, 0], [0.8, -0.4, 0.3, -1.0], [0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0], [0, 0.8, 0.8], [0, 0.2, 0.3]], [[0, 0, 0, 0], [0.6, 0.3, 0.5, 0.4], [0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0], [0, 0.3, 0.3], [0, 0.4, 0.6]], [[0, 0, 0, 0], [0.2, 0.4, 0.6, 0.3], [0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0], [0, -0.7, -0.8], [0.7, 0, 0.1]]], [[[1.0, 0.8, 0.6, 0.2], [0, 0, 0, 0], [0.4, 0, 0, 0, 0.9], [0, 0, 0, 0.9], [0.5, 0.8], [0.8, 0.4, 0.5], [0.6, 0.2, 0.7]], [[-1.0, -0.4, 0.3, 0.4], [0, 0, 0, 0], [0, 0.8, 0.8, 0.9, -0.4], [0, 0.8, -0.1, 0], [0.2, 0.7], [-0.3, -1.0, -1.0], [-0.3, 0.9, -0.6]], [[-1.0, 0.3, 0.5, 0.6], [0, 0, 0, 0], [0.8, 0.4, 0.2, 0, 0.6], [0, 0, 0, 0], [-0.8, 0.7], [0.5, -0.7, -1.0], [0.8, -0.1, 0.2]], [[-1.0, -1.0, 0.4, 0.3], [0, 0, 0, 0], [0.1, 0, 0, 0, 0.4], [0.6, 0, 0, -0.6], [-1.0, 1.0], [-0.9, -1.0, -1.0], [-1.0, 0.6, -1.0]]], [[[0, 0, 0, 0], [0.4, 0, 0.8, 0.

In [16]:
prior_probs2 = calculate_efficiency(post_probs1, 'efficiency')

Interstage correlations:  [[[[0, 0, 0, 0], [0.7, -1.0, -1.0, -1.0, -1.0, 0.5, -1.0, -1.0, -0.4, -1.0], [0, 0, 0], [0.5, 0.6, 0.4], [0.7, 0.8, 0.3, -0.2], [0, 0, -0.6, 0]], [[0, 0, 0, 0], [0.8, 0.8, 0.9, 0.6, 0.4, 0.9, 0.5, 0.2, -0.1, 0.9], [0, 0, 0], [0.4, 0.5, 0.5], [0.7, 0.8, 0.5, -0.1], [0, 0.6, 0, 0]], [[0, 0, 0, 0], [0.8, 0.5, 0.8, 0.9, 0.6, 0.8, 0.7, 0.5, 0.2, 0.7], [0, 0, 0], [0.4, 0.5, 0.5], [0.6, 0.4, 0.7, 0.1], [0, 0.4, 0, 0]], [[0, 0, 0, 0], [0.6, 0.4, 0.6, 0.5, 0.8, 0.7, 0.9, 0.7, 0.3, 0.5], [0, 0, 0], [0.8, 0.3, 0.2], [0.4, 0.2, 0.7, 0.4], [0, 0, 0, 0]]], [[[-0.2, 0.4, 0.5, 0.3], [0.9, 0.9, 0.6, -0.7, -0.4, 0.7, -0.5, -0.6, 0.7, 0.4], [0.2, 0.5, 0.6], [0.5, 0.7, 0.6], [0.7, 0.6, 0.8, -0.1], [0.5, 0.4, 0.6, 0.5]], [[0.8, 0.3, -0.6, -1.0], [-0.8, -0.8, 0.5, -0.2, 0.4, 0.3, 0.8, 0.7, -0.4, -1.0], [0.1, 0.7, 0.2], [0.8, -0.2, -0.4], [-0.1, -0.4, 0.8, 0.6], [0.8, -0.3, -0.6, 0.2]], [[-0.1, 0.6, 0.5, 0.2], [-0.7, -0.8, 0.8, 0.2, 0.6, 0.5, 0.9, 0.6, 0.1, -0.4], [0.4, 0.4, 0.8], [

In [19]:
post_probs2 = calculate_second_stage(prior_probs2)

Correlations:  [[[[0, 0, 0, 0], [0.4, 0.4, -0.3, -0.1, 0.2, -0.8, -0.7, 0.6, 0.1, -0.3], [0.9, 0.3, -0.4], [0.9, -0.7, -0.8], [0, 0, 0.9, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0.9, 0.8, -0.2, 0.1, 0.7, 0.1, 0.2, 0.8, 0.6, 0.1], [0.5, 0.7, 0.1], [0.6, 0.3, 0.2], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [0.3, 0.2, 0.8, 0.5, 0.1, 0.6, 0.8, 0.3, 0.5, 0.7], [0.3, 0.6, 0.5], [0.6, 0.5, 0.4], [0.6, 0.7, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 0], [-0.8, -0.9, 0.6, 0.2, -0.7, 0.4, 0.3, -0.5, -0.1, 0.5], [0.2, 0.6, 0.8], [0.5, 0.7, 0.6], [0.5, 0.6, 0, 0], [0, 0, 0, 0]]], [[[0.4, 0.9, 0.3, -0.8], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0], [0.4, 0, 0], [-0.4, -0.7, 0.8, 0.1], [0.8, 0.2, 0.3, 0.4]], [[0.4, 0.8, 0.2, -0.9], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0], [0.5, 0, 0], [-0.9, -0.9, 0.7, 0.4], [0.7, 0.1, 0.2, 0.7]], [[-0.3, -0.2, 0.8, 0.6], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0], [0, 0.2, 0.1], [0.4, -0.6, 0.6, -0.1], [0.7, 0.1, 0.4, 0.6]], [[-0.1, 0.1, 0.5, 0.2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0